In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

True

In [41]:
sys.path.insert(0, "../..")

In [42]:
from comms import telegram
from agent_tools import pinecone

In [43]:
system_message = Path("system_message_jaded_rose_chatbot.txt").read_text()
#system_message

In [44]:
msg = telegram.trigger({
    "bot_token": os.getenv("TELEGRAM_BOT_TOKEN"),
    "include_existing": False,
    "timeout_seconds": 120
})
#msg

In [45]:
prompt = msg["text"]
#prompt

In [46]:
index = pinecone.get_index(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    api_key=os.getenv("PINECONE_API_KEY"),
)
chunks = pinecone.query_chunks(
    index=index,
    prompt=prompt,
    namespace=os.getenv("PINECONE_NAMESPACE"),
    
)
#chunks

In [47]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "system", "content": f"Retrieved context:\n\n" + "\n\n".join(c["text"] for c in chunks)},
        {"role": "user", "content": msg["text"]},
    ],
)

answer = response.choices[0].message.content
answer

"I'm sorry to hear that your item is damaged. Please inspect your order and contact us immediately if the item is defective, damaged, or if you received the wrong item. This way, we can resolve the issue accordingly. If you need further assistance, please email us at hello@jadedrose.co.uk."